In [92]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from fake_useragent import UserAgent
import re
import pandas as pd
import numpy as np

# Some settings

In [132]:
binary_components_path = "D:/IT/Data_science/lessons/27. lesson 27.02/selenium/Win_x64_1265703_chrome-win/chrome-win/chrome.exe"

driver_path = "D:/IT/Data_science/lessons/27. lesson 27.02/selenium/Win_x64_1265703_chromedriver_win32/chromedriver_win32/chromedriver.exe"

options = webdriver.ChromeOptions()
options.binary_location = binary_components_path
service = Service(driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.glassdoor.com/Job/index.htm")

In [133]:
df = pd.DataFrame(columns=['employer', 'employer_rating', 'sector', 'industry', 'employer_founded', 'employer_revenue',  'job_title', 'location',
                           'min_salary', 'max_salary', 'average_base_salary', 'per_hour_or_year'])

# Function for getting links to vacancies

In [134]:
def get_unique_job_links():
    job_links = set()

    while True:
        show_more_counter = 0
        while show_more_counter < 2:
            show_more_button = driver.find_element(By.XPATH, '//*[@id="left-column"]/div[2]/div/button') 
            
            if show_more_button.is_displayed():
                # Scroll down the page to the button "Show More"
                driver.execute_script("arguments[0].scrollIntoView();", show_more_button)
                sleep(2)
                # Wait until the button becomes clickable
                WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="left-column"]/div[2]/div/button')))
                
                show_more_button.click()
                sleep(4)

                try:
                    close_button = driver.find_element(By.CLASS_NAME, value='CloseButton')
                    close_button.click()
                except:
                    pass

                show_more_counter += 1

            else:
                break

        job_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_trackingLink__GrRYn')
        for job_element in job_elements:
            job_link = job_element.get_attribute('href')
            if job_link not in job_links:
                job_links.add(job_link)
        else:
            break
    
    return job_links

# Insert some text

In [135]:
search_box = driver.find_element(By.XPATH, '//*[@id="searchBar-jobTitle"]')
search_box.send_keys('Data Scientist')
search_box.submit()

This was the first code to get links. This code is commented out because it will overwrite the txt file with links.

In [130]:
# with open('job_links.txt', 'w') as file:
#     job_links = get_unique_job_links()

#     for link in job_links:
#         file.write(link + '\n')
        
# print("Links to vacancies are saved to a file job_links.txt")
# print(len(job_links))


#driver.quit()

# Add more job links

In [124]:
# Open the file and read the existing links
with open('job_links.txt', 'r') as file:
    existing_links = set(line.strip() for line in file)

# Open the file to add new links
with open('job_links.txt', 'a') as file:
    # Get new unique links to vacancies
    new_links = get_unique_job_links()

    # Filter only those links that are not yet in the existing list
    unique_new_links = new_links - existing_links

    # Add new unique links to vacancies
    for link in unique_new_links:
        file.write(link + '\n')

print("New job links added to the file job_links.txt")

New job links added to the file job_links.txt


# Check if there are any duplicates

In [136]:
# Open the file and read all the lines
with open('job_links.txt', 'r') as file:
    lines = file.readlines()

# Remove unnecessary characters such as spaces and newline characters
cleaned_lines = [line.strip() for line in lines]

# Check for duplicates
if len(cleaned_lines) == len(set(cleaned_lines)):
    print("There are no duplicates")
else:
    print("There are duplicates")

There are no duplicates


# Function to extract the text of an element

In [137]:
def get_element_text(driver, by, value):
    try:
        element = driver.find_element(by, value)
        if element.text == '--':
            return np.nan
        else:
            return element.text
    except NoSuchElementException:
        return np.nan

# Function for parsing existing job links

In [138]:
def parse_jobs(job_links, driver):
    count = 0
    ua = UserAgent()
    driver = driver

    count= int(count)
    for i, job_link in enumerate(job_links, start=1):
        i = int(i)
        if count >= 1500:
            break

        if i% 4 == 0:
            if driver:
                driver.quit()
            
            new_user_agent = ua.random
            options = webdriver.ChromeOptions()
            options.add_argument(f'user-agent={new_user_agent}')
            options.binary_location = binary_components_path
            service = Service(driver_path)
            driver = webdriver.Chrome(service=service, options=options)

        try:
            driver.get(job_link)
            sleep(4)
            

            employer = get_element_text(driver, By.CLASS_NAME, 'EmployerProfile_employerName__qujuA')
            employer_rating = get_element_text(driver, By.CLASS_NAME, 'EmployerProfile_ratingContainer__ul0Ef')
            sector = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[2]/div/div/div[5]/div')
            industry = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[2]/div/div/div[4]/div')
            employer_founded = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[2]/div/div/div[2]/div')
            employer_revenue = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[2]/div/div/div[6]/div')
            job_title = get_element_text(driver, By.CLASS_NAME, 'JobDetails_jobTitle__Xvsha')
            location = get_element_text(driver, By.CLASS_NAME, 'JobDetails_location__mSg5h')
            min_salary = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[1]/div/div/div[2]/div[3]/div[1]')
            max_salary = get_element_text(driver, By.XPATH, '//*[@id="app-navigation"]/div[3]/div/div[1]/div/div[1]/div/section/section[1]/div/div/div[2]/div[3]/div[2]')
            # In order to separate text from a number and take into account the appearance of both hourly and annual salaries
            annual_salary_pattern = r'\d+(\.\d+)?K'
            hourly_salary_pattern = r'\d+(\.\d+)?'
            min_salary = re.search(annual_salary_pattern if 'K' in min_salary else hourly_salary_pattern, min_salary).group()
            max_salary = re.search(annual_salary_pattern if 'K' in max_salary else hourly_salary_pattern, max_salary).group()

            average_base_salary = get_element_text(driver, By.CLASS_NAME, 'SalaryEstimate_averageEstimate__xIgkL')
            per_hour_or_year = get_element_text(driver, By.CLASS_NAME, 'SalaryEstimate_payPeriod__RsvG_') 
            
            
            new_row = {'employer': employer,
                       'employer_rating': employer_rating,
                       'sector': sector,
                       'industry': industry,
                       'employer_founded': employer_founded,
                       'employer_revenue': employer_revenue,
                       'job_title': job_title,
                       'location': location,
                       'min_salary': min_salary,
                       'max_salary': max_salary,
                       'average_base_salary': average_base_salary,
                       'per_hour_or_year': per_hour_or_year}
            df.loc[len(df)] = new_row
            
            count += 1
            print(f"The vacancies counter: {count}")
            sleep(5)
        except Exception as e:
            print(f"An error occurred while processing the link {job_link}: {str(e)}")
            continue

    if driver:
        driver.quit()
    
    df.to_csv('auto_data.csv', index=False)

# Parse job links

In [139]:
with open('job_links.txt', 'r') as file:
    job_links = [line.strip() for line in file.readlines()]

parse_jobs(job_links, driver)

The vacancies counter: 1
The vacancies counter: 2
The vacancies counter: 3
The vacancies counter: 4
The vacancies counter: 5
An error occurred while processing the link https://www.glassdoor.com/partner/jobListing.htm?pos=214&ao=1136043&s=58&guid=0000018e00fc65a7b35eec6f0cde29f3&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_74377381&cb=1709413525490&jobListingId=1009013278663&jrtk=5-yul1-0-1ho0fopfhi6km800-7b06bbc430b58065: argument of type 'float' is not iterable
The vacancies counter: 6
The vacancies counter: 7
The vacancies counter: 8
The vacancies counter: 9
The vacancies counter: 10
The vacancies counter: 11
The vacancies counter: 12
The vacancies counter: 13
The vacancies counter: 14
The vacancies counter: 15
The vacancies counter: 16
The vacancies counter: 17
The vacancies counter: 18
The vacancies counter: 19
The vacancies counter: 20
The vacancies counter: 21
The vacancies counter: 22
The vacancies counter: 23
The vacancies counter: 24
The vacancies counter: 25
The vacancies counter: 26
T